In [65]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [3]:
#pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [4]:
data = pd.read_csv('clear4.csv', encoding = 'utf-8')
new = data[['in_out','text']].copy()    
df_in = new.loc[new['in_out'] == 'in']
df_out = new.loc[new['in_out'] == 'out']
df_hold = new.loc[new['in_out'] == 'hold'] 

In [5]:
print(data.shape)
print(new.shape)
print(df_in.shape)
print(df_out.shape)
print(df_hold.shape)

(6612, 31)
(6612, 2)
(2625, 2)
(2963, 2)
(1024, 2)


In [45]:
df_in.head(3)

,in_out,text
0,in,"""Учитель для России"" для меня возможность полу..."
1,in,"Я хочу стать участником программы, потому что ..."
2,in,"Начну с того, что ещё в школе я смотрела на св..."


## Токенизация, удаление стоп-слов и нормализация.

In [34]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [10]:
df_in["text_norm"] = df_in["text"].apply(normalize)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_out['text_norm'] = df_out['text'].apply(normalize)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df_hold['text_norm'] = df_hold['text'].apply(normalize)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
new['text_norm'] = new['text'].apply(normalize)

Попробуем те же самые методы.

Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [56]:
df_in['text_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [школа, образование, работать, хотеть, учитель...
1     [который, хотеть, учитель, стать, это, человек...
2     [новый, это, учитель, школа, который, опыт, хо...
3     [учить, хотеть, наш, год, язык, обучение, урок...
4     [ребёнок, работа, попробовать, изменить, друг,...
5     [человек, предмет, это, экономический, ребёнок...
6     [программа, учитель, который, работа, являться...
7     [ребёнок, который, школа, программа, жизнь, ин...
8     [учитель, ребёнок, стать, школа, это, учиться,...
9     [психологический, развитие, мочь, ребёнок, род...
10    [друг, хотеть, любить, язык, стать, участник, ...
11    [программа, опыт, почему, направить, хотеть, о...
12    [учитель, стать, учить, тот, помочь, один, жиз...
13    [коллега, стать, учитель, программа, педагогич...
14    [который, программа, другой, учитель, стать, с...
15    [ребёнок, работа, учитель, курс, это, опыт, зн...
16    [который, год, студент, 2015, ребёнок, это, да...
17    [ребёнок, взрослый, знание, психологически

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [16]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [35]:
df_in["text_norm"] = df_in["text"].apply(normalize)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
kw_in = []
for i in df_in['text_norm']:
    for n in i:
        kw_in.append(n)

In [37]:
df_out['text_norm'] = df_out['text'].apply(normalize)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
kw_out = []
for i in df_out['text_norm']:
    for word in i:
        kw_out.append(word)

In [39]:
df_hold['text_norm'] = df_hold['text'].apply(normalize)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
kw_hold = []
for i in df_hold['text_norm']:
    for word in i:
        kw_hold.append(word)

In [49]:
new['text_norm'] = new['text'].apply(normalize)

In [51]:
kw_all = []
for i in new['text_norm']:
    for word in i:
        kw_all.append(word)

In [52]:
kw = pd.DataFrame({
    'in_out': ['in', 'out', 'hold','all'],
    'keywords': [kw_in, kw_out, kw_hold,kw_all]})

Ещу улучшения!

In [53]:
kw['keywords'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(4)

0    [ребёнок, школа, это, учитель, который, свой, ...
1    [ребёнок, школа, это, учитель, свой, хотеть, к...
2    [ребёнок, школа, это, учитель, свой, хотеть, к...
3    [ребёнок, школа, это, учитель, свой, который, ...
Name: keywords, dtype: object

In [61]:
kw

,in_out,keywords,text_norm_str
0,in,"[учитель, россия, возможность, получить, актуа...",учитель россия возможность получить актуальный...
1,out,"[32, год, прошлое, год, наконец-то, получить, ...",32 год прошлое год наконец-то получить высокий...
2,hold,"[хотеть, стать, участник, программа, учитель, ...",хотеть стать участник программа учитель россия...
3,all,"[учитель, россия, возможность, получить, актуа...",учитель россия возможность получить актуальный...


Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [54]:
kw['text_norm_str'] = kw['keywords'].apply(' '.join)

In [68]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3)

In [71]:
tfidf.fit(kw['text_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [72]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [73]:
texts_vectors = tfidf.transform(kw['text_norm_str'])

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return X


Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [74]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-20:-1]] 

In [75]:
keywords[:4]

[['ребёнок',
  'школа',
  'это',
  'учитель',
  'который',
  'свой',
  'хотеть',
  'программа',
  'работа',
  'стать',
  'работать',
  'человек',
  'мой',
  'образование',
  'знание',
  'ученик',
  'возможность',
  'очень',
  'жизнь'],
 ['ребёнок',
  'школа',
  'это',
  'учитель',
  'свой',
  'хотеть',
  'который',
  'работа',
  'программа',
  'знание',
  'стать',
  'работать',
  'человек',
  'опыт',
  'образование',
  'учиться',
  'очень',
  'возможность',
  'мой'],
 ['ребёнок',
  'школа',
  'это',
  'учитель',
  'свой',
  'хотеть',
  'который',
  'программа',
  'работа',
  'человек',
  'стать',
  'работать',
  'учиться',
  'очень',
  'знание',
  'образование',
  'мой',
  'жизнь',
  'ученик'],
 ['ребёнок',
  'школа',
  'это',
  'учитель',
  'свой',
  'который',
  'хотеть',
  'программа',
  'работа',
  'стать',
  'работать',
  'человек',
  'знание',
  'мой',
  'образование',
  'очень',
  'учиться',
  'возможность',
  'жизнь']]

In [ ]:
#evaluate(data['keywords'], keywords)

Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [106]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [107]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [109]:
%%time
keywords_rw = kw['keywords'].apply(lambda x: get_kws(x, 10, 10))

Wall time: 4.49 s


In [ ]:
#evaluate(data['keywords'], keywords_rw)

In [110]:
keywords_rw.head(10)

0    [ребёнок, учитель, школа, работа, программа, п...
1    [ребёнок, учитель, школа, опыт, работа, жизнь,...
2    [ребёнок, учитель, школа, человек, жизнь, врем...
Name: keywords, dtype: object

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [111]:
import networkx as nx

In [112]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [113]:
%%time
keyword_nx = kw['keywords'].apply(lambda x: some_centrality_measure(x, 10, 10))

AttributeError: module 'networkx' has no attribute 'from_numpy_array'

In [ ]:
evaluate(data['keywords'], keyword_nx)

Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/Zb89yjXFr37EITMq1

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другой.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 0.5 баллов).